### Import Data & Modules

In [1]:
import pandas as pd
import numpy as np

import pickle
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path

from src import data_prep 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)


df_resampled = data_prep.make_dataframe()

# ADDING SENTIMENT AND CATEGORY COLUMNS
with open("data/sentiment_columns.pkl", "rb") as f:
        sentiment_columns = pickle.load(f)

with open("data/category_columns_6.pkl", "rb") as f:
        category_columns = pickle.load(f)


#with open("data/embedding_columns_5.pkl", "rb") as f:
#        embedding_columns = pickle.load(f)


df_resampled = df_resampled.merge(sentiment_columns, on="new_id", how="left")
df_resampled = df_resampled.merge(category_columns, on="new_id", how="left")
#df_resampled = df_resampled.merge(embedding_columns, on="new_id", how="left")

df_resampled['predicted_product_category'].unique()


df_major_category = (
    df_resampled
    .groupby(["id", "predicted_product_category"])
    .size()
    .reset_index(name="count")
)

# Pick the category with the highest count for each id
df_major_category = (
    df_major_category
    .sort_values(["id", "count"], ascending=[True, False])
    .groupby("id")
    .first()
    .reset_index()
    .rename(columns={"predicted_product_category": "final_category"})
)


df_resampled = df_resampled.merge(df_major_category[["id", "final_category"]], on="id", how="left")
df_resampled['predicted_product_category'] = df_resampled['final_category']

df_view = df_resampled.drop(columns=["sourceURLs", "imageURLs", "keys"])
df_view.shape

(61086, 29)

In [ ]:
# df_id_category_counts = (
#     df_view.groupby(["id", "final_category"])
#            .size()
#            .reset_index(name="count")
#            .groupby("id")
#            .apply(lambda x: dict(zip(x["final_category"], x["count"])))
#            .reset_index(name="category_counts")
# )

# df_id_category_counts = (
#     df_view.groupby(["id", "name"])
#            .size()
#            .reset_index(name="count")
#            .groupby("id")
#            .apply(lambda x: dict(zip(x["name"], x["count"])))
#            .reset_index(name="category_counts")
# )



# df_id_category_counts

In [ ]:

#df_view.tail(100)


df_view[df_view['asins']=='B00QWO9P0O,B01IB83NZG,B00MNV8E0C	']#.head(100)



df_view[df_view['id']=='AVpgjNRkilAPnD_xuALS']

#df_view.predicted_product_category.value_counts()

#df_view['id'].value_counts()

#df_view.reviews.rating.value_counts()
#df_view['id'].value_counts()
#df_view['name'].value_counts()

#df_resampled[df_resampled['id']=='AV1YE_muvKc47QAVgpwE'].head(100)


#df_view[df_view['primaryCategories']== 'Health & Beauty'].head(1)

#df_view.head(1)

#### Pivots of Best and Worst Products

In [2]:

out = (
    df_resampled
    .groupby(["predicted_product_category", "id"])
    .agg(
        # existing
        count_new_id = ("new_id", "count"),
        count_positive = ("sentiment", lambda x: (x == "positive").sum()),
        count_negative = ("sentiment", lambda x: (x == "negative").sum()),
        count_neutral  = ("sentiment", lambda x: (x == "neutral").sum()),

        # new fields
        count_didPurchase  = ("reviews.didPurchase", "sum"),
        count_doRecommend  = ("reviews.doRecommend", "sum"),
        sum_numHelpful     = ("reviews.numHelpful", "sum"),
        sum_rating         = ("reviews.rating", "mean"),    

        #name_first = ("name", "first"),
        name = ("name", lambda x: x.value_counts().idxmax()),
        imageURLs = ("imageURLs", lambda x: x.dropna().value_counts().idxmax() if x.dropna().size > 0 else None),

    )
    .reset_index()
)

out = out[(out['count_new_id'] >= 5)].reset_index(drop=True)

out["count_new_id_by_category"] = (
    out.groupby("predicted_product_category")["count_new_id"]
       .transform("sum")
)

out["count_do_recommend_by_category"] = (
    out.groupby("predicted_product_category")["count_doRecommend"]
       .transform("sum")
).replace(0, 1)

out['positive_sentiment_ratio'] = out['count_positive'] / out['count_new_id']
out['negative_sentiment_ratio'] = out['count_negative'] / out['count_new_id']
out['neutral_sentiment_ratio'] = out['count_neutral'] / out['count_new_id']

out['sentiment_score'] = out['positive_sentiment_ratio'] - out['negative_sentiment_ratio']
out['rating_score'] = out['sum_rating']/5

out['frequency_score'] = out['count_new_id'] / out['count_new_id_by_category']

out['recommendation_score'] = out['count_doRecommend'] / out['count_do_recommend_by_category']


out['total_score_1'] =0.35 * out['sentiment_score'] + 0.35 * out['rating_score'] + 0.15 * out['frequency_score'] + 0.15 * out['recommendation_score']
out['total_score_2'] =0.4 * out['sentiment_score'] + 0.4 * out['rating_score'] + 0 * out['frequency_score'] + 0.2 * out['recommendation_score']

out["best_rank_in_category"] = (
    out.groupby("predicted_product_category")["total_score_1"]
      .rank(method="dense", ascending=False).astype(int)
)

out["worst_rank_in_category"] = (
    out.groupby("predicted_product_category")["total_score_1"]
      .rank(method="dense", ascending=True).astype(int)
)

front_cols = ['best_rank_in_category', 'worst_rank_in_category', 'total_score_1', 'total_score_2']
cols = front_cols + [col for col in out.columns if col not in front_cols]
out = out[cols].sort_values(by=['predicted_product_category','total_score_1'], ascending=False).reset_index(drop=True)
out['category_id'] = out['predicted_product_category'] + '_' + out['id']

best_products = out[out['best_rank_in_category']<=3].sort_values(by=['predicted_product_category','best_rank_in_category'], ascending=True).reset_index(drop=True)
worst_products = out[out['worst_rank_in_category']<=3].sort_values(by=['predicted_product_category','worst_rank_in_category'], ascending=True)
worst_products = worst_products[~worst_products['category_id'].isin(best_products['category_id'])].reset_index(drop=True)



### Adding the Reviews to Best and Worst Products

In [3]:
df_summary = df_resampled.copy()

df_summary['category_id'] = df_summary['predicted_product_category'] + '_' + df_summary['id']

df_summary_best = df_summary[df_summary['category_id'].isin(best_products.category_id.unique())]
df_summary_worst = df_summary[df_summary['category_id'].isin(worst_products.category_id.unique())]


def make_summary_strings(products_df, 
summary_df, 
positive_label="positive", 
negative_label="negative"):


    summary_strings = []

    for row in products_df.itertuples(index=False):
        col1 = row.category_id

        # Sentiment proportions
        col2 = row.positive_sentiment_ratio / (
            row.positive_sentiment_ratio + row.negative_sentiment_ratio
        )
        col3 = row.negative_sentiment_ratio / (
            row.positive_sentiment_ratio + row.negative_sentiment_ratio
        )

        # Get positive subset
        df_subset_pos = (
            summary_df
            .loc[(summary_df['category_id'] == col1) &
                 (summary_df['sentiment'] == positive_label)]
            .sort_values(by='reviews.numHelpful', ascending=False)
            .head(int(20 * col2))
        )

        # Get negative subset
        df_subset_neg = (
            summary_df
            .loc[(summary_df['category_id'] == col1) &
                 (summary_df['sentiment'] == negative_label)]
            .sort_values(by='reviews.numHelpful', ascending=False)
            .head(int(20 * col3))
        )

        # selected indexes
        selected_idx = list(df_subset_pos.index) + list(df_subset_neg.index)

        # extract & join text
        review_texts = summary_df.loc[selected_idx, "name_title_text"].tolist()
        summary_string = " ".join(review_texts)

        summary_strings.append(summary_string)

    # Add new column (name auto-handled)
    products_df["summary_reviews_string"] = summary_strings

    cols = ['category_id', 'predicted_product_category', 'id', 'count_new_id',  'name', 'imageURLs', 'summary_reviews_string']
    products_df = products_df[cols]


    return products_df

best_products = make_summary_strings(best_products, df_summary_best)
worst_products = make_summary_strings(worst_products, df_summary_worst)


with open(Path("data/best_products.pkl"), "wb") as f:
    pickle.dump(best_products, f)

with open(Path("data/worst_products.pkl"), "wb") as f:
    pickle.dump(worst_products, f)

### Final Products

In [4]:
#from src import model3

# with open(Path("data/best_products_with_llm_summary.pkl"), "rb") as f:
#     best_products_with_llm_summary= pickle.load(f)

# with open(Path("data/worst_products_with_llm_summary.pkl"), "rb") as f:
#     worst_products_with_llm_summary = pickle.load(f)


# cols = ['predicted_product_category', 'name', 'imageURLs', 'llm_summary']

# best_products_with_llm_summary = best_products_with_llm_summary[cols]
# worst_products_with_llm_summary = worst_products_with_llm_summary[cols]

# json_best_data = best_products_with_llm_summary.to_json(orient="records")
# json_worst_data = worst_products_with_llm_summary.to_json(orient="records")


with open(Path("data/json_best_data.pkl"), "rb") as f:
    json_best_data = pickle.load(f)

with open(Path("data/json_worst_data.pkl"), "rb") as f:
    json_worst_data = pickle.load(f)



# with open(Path("data/best_products.pkl"), "wb") as f:
#     pickle.dump(best_products, f)

# with open(Path("data/worst_products.pkl"), "wb") as f:
#     pickle.dump(worst_products, f)

In [16]:
import json
data = json.loads(json_best_data)
df_best = pd.DataFrame(data)
with open("data/best_products.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

In [14]:
json_best_data

'[{"predicted_product_category":" Batteries","name":"AmazonBasics AAA Performance Alkaline Batteries (36 Count)","imageURLs":"https:\\/\\/images-na.ssl-images-amazon.com\\/images\\/I\\/81qmNyJo%2BkL._SL1500_.jpg,https:\\/\\/images-na.ssl-images-amazon.com\\/images\\/I\\/81ZTa1BrkzL._SL1500_.jpg,http:\\/\\/ecx.images-amazon.com\\/images\\/I\\/41pSoRu7UlL._SS40_.jpg,https:\\/\\/www.upccodesearch.com\\/images\\/barcode\\/0841710106442.png,https:\\/\\/images-na.ssl-images-amazon.com\\/images\\/I\\/81qmNyJo%252525252BkL._SL1500_.jpg,https:\\/\\/images-na.ssl-images-amazon.com\\/images\\/I\\/51DfBuqy4qL._SL1189_.jpg,https:\\/\\/images-na.ssl-images-amazon.com\\/images\\/I\\/51TshsEJTIL.jpg,https:\\/\\/images-na.ssl-images-amazon.com\\/images\\/I\\/81qmNyJo%2525252525252BkL._SL1500_.jpg,https:\\/\\/images-na.ssl-images-amazon.com\\/images\\/I\\/81qmNyJo%25252BkL._SL1500_.jpg,https:\\/\\/images-na.ssl-images-amazon.com\\/images\\/I\\/81qmNyJo%2525252BkL._SL1500_.jpg,https:\\/\\/images-na.ssl-i